In [1]:
import numpy as np
import ObjectiveFunctions as of
import OptiPlot as op
import BevinsCuckooSearch as bcs
import ContinuousGnowee as cg
import YangCuckooSearch as ycs
import ModifiedCuckooSearch as mcs
import DiscreteGnowee as dg
import DiscreteCuckooSearch as dcs 
import ZhouDiscreteCuckooSearch as zdcs 
import BevinsDiscreteCuckooSearch as bdcs 
import Gnowee
import CSLibrary as csl
import Utilities as util
import math as m
from operator import attrgetter

# Select optimization problem type and associated parameters
#opt_funct=of.WeldedBeam_Obj
#opt_funct=of.PressureVessel_Obj
#opt_funct=of.SpeedReducer_Obj
#opt_funct=of.Spring_Obj
#opt_funct=of.MI_Spring_Obj
#opt_funct=of.MI_PressureVessel_Obj
opt_funct=of.Chemical_Process_Obj

#opt_funct=of.Ackley_Obj
#opt_funct=of.Shifted_Ackley_Obj
#opt_funct=of.DeJong_Obj
#opt_funct=of.Shifted_DeJong_Obj
#opt_funct=of.Easom_Obj
#opt_funct=of.Shifted_Easom_Obj
#opt_funct=of.Griewank_Obj
#opt_funct=of.Shifted_Griewank_Obj
#opt_funct=of.Rastrigin_Obj
#opt_funct=of.Shifted_Rastrigin_Obj
#opt_funct=of.Rosenbrock_Obj
#opt_funct=of.Shifted_Rosenbrock_Obj

#opt_funct=of.TSP_Obj

# Set optimization settings
opt_params=of.Get_Params(opt_funct,'Gnowee',dimension=7)
S=csl.Settings(optimal_fitness=opt_params.o)

# Initialize variables
max_iter=100#Number of algorithm iterations
eval_interval=500 # Fuction eval interval at which the fitness is sampled.  
history=[]   #List that contains the final timeline results from each optimization run 
# History of fitness vs function evals  
feval_history=np.array([[i*eval_interval,0.0,0.0,0] for i in \
                            range(int(S.em/eval_interval)+1)])     #[Feval, Fit avg, Fit std, counter]

for i in range(0,max_iter,1):
    print "Run #%d" %i
    S.d=False
    
    YCS_on=False
    MCS_on=False
    BCS_on=False
    CG_on=False
    DCS_on=False
    ZDCS_on=False
    BDCS_on=False
    DG_on=False
    G_on=True#False

    # Gnowee
    if G_on:
        import GnoweeUtilities as util
        opt_params=of.Get_Params(opt_funct,'Gnowee',dimension=7)
        S=util.Settings(optimal_fitness=opt_params.o)
        S.s='lhc'
        S.p=25
        S.a=0.5
        S.fd=0.2
        S.fe=0.2
        S.fl=0.2
        S.sl=225
        S.sf=10
        S.em=200000
        (timeline)=Gnowee.main(opt_funct,opt_params.lb,opt_params.ub,opt_params.vt,S,discreteVals=opt_params.dv)
        
    #YCS
    if YCS_on:
        #S.em=10000
        (timeline)=ycs.CS(opt_funct,opt_params.lb,opt_params.ub,S)

    # BCS
    if BCS_on:
        S.c=25
        S.a=1.5
        S.pa=0.25
        S.pt=0.04
        S.s='lhc'
        S.sf=10
        #S.gm=20000
        S.sl=400
        #S.em=200000
        #S.ot=0.01
        (timeline)=bcs.CS(opt_funct,opt_params.lb,opt_params.ub,S)

    # CG
    if CG_on:
        S=util.Settings(optimal_fitness=opt_params.o)
        S.p=25
        S.a=0.5
        S.fd=0.2
        S.fe=0.1
        S.fl=0.2
        S.s='lhc'
        S.sl=400
        S.sf=10
        S.em=200000
        (timeline)=cg.Gnowee(opt_funct,opt_params.lb,opt_params.ub,S)
    
    # MCS
    if MCS_on:
        S.pa=0.75
        S.pt=0.25
        S.sl=500
        S.s='lhc'
        S.c=20
        (timeline)=mcs.MCS(opt_funct,opt_params.lb,opt_params.ub,S)

    # DCS
    if DCS_on:
        S.pa=0.2
        S.pt=0.6
        S.c=20
        S.gm=15000
        S.sl=400
        tsp_prob=dcs.Read_TSP('TSPLIB\eil51.tsp')
        S.of=426.0
        (timeline)=dcs.DCS(opt_funct,S,tsp_prob)
        
    # ZDCS
    if ZDCS_on:
        S.pa=0.5
        S.c=100
        S.a=2
        S.gm=40
        S.sl=10
        S.em=200000
        tsp_prob=dcs.Read_TSP('TSPLIB\eil51.tsp')
        S.of=426.0
#        tsp_prob=dcs.Read_TSP('TSPLIB\st70.tsp')
#        S.of=675.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\pr107.tsp')
#        S.of=44303.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\\bier127.tsp')
#        S.of=118282.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\\ch150.tsp')
#        S.of=6528.0
        (timeline)=zdcs.DCS(opt_funct,S,tsp_prob)
        
    # DMC
    if BDCS_on:
        S.pa=0.2
        S.pt=0.2
        S.c=40
        S.a=2        
        S.gm=4000000
        S.ot=0.0001
        S.sl=250000
        S.em=100000
#        tsp_prob=dcs.Read_TSP('TSPLIB\eil51.tsp')
#        S.of=426.0
#        tsp_prob=dcs.Read_TSP('TSPLIB\st70.tsp')
#        S.of=675.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\pr107.tsp')
#        S.of=44303.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\\bier127.tsp')
#        S.of=118282.0
#        tsp_prob=zdcs.Read_TSP('TSPLIB\\ch150.tsp')
#        S.of=6528.0
        (timeline)=bdcs.DCS(opt_funct,S,tsp_prob)
        
    # Discrete Gnowee
    if DG_on:
        S=util.Settings(optimal_fitness=opt_params.o)
        S.s='nolh-cdr'
        S.p=25
        S.a=0.5
        S.fd=0.2
        S.fe=0.1
        S.fl=0.2  
        S.gm=650
        S.sl=50
        S.em=200000
        tsp_prob=dg.Read_TSP('TSPLIB\eil51.tsp')
        S.of=426.0
#        tsp_prob=dg.Read_TSP('TSPLIB\st70.tsp')
#        S.of=675.0
#        tsp_prob=dg.Read_TSP('TSPLIB\pr107.tsp')
#        S.of=44303.0
#        tsp_prob=dg.Read_TSP('TSPLIB\\bier127.tsp')
#        S.of=118282.0
#        tsp_prob=dg.Read_TSP('TSPLIB\\ch150.tsp')
#        S.of=6528.0
        (timeline)=dg.Gnowee(opt_funct,S,tsp_prob)
    
#    (feval,gens,out_data)=run.DMC(opt_funct,opt_params.lb,opt_params.ub,pa=0.25,pc=0.75,frac_top=0.25,cuckoos=25,max_gens=20000,
#    feval_max=200000,conv_tol=1E-5,stall_iter_limit=40000,optimal_fitness=global_opt,opt_conv_tol=1E-2,
#    mutate='distance method')

    # Ouput problem outputs to user
    debug=False
    if debug==True:
        print "The best design is", timeline[-1].d," with a fitness of ", timeline[-1].f 
        print "The number of generations was %d with %f objective function evals." %(timeline[-1].g,timeline[-1].e)
    
    # Save final timeline data for future processing
    mingen = min(timeline,key=attrgetter('f'))
    history.append(csl.Event(mingen.g,mingen.e,mingen.f,mingen.d))

    # Update Fitness vs Feval History
    k=1 
    
    # Compute the average and standard deviation using a recurrence relation
    for j in range(0,len(feval_history),1):
        while timeline[k].e < feval_history[j,0] and k+1<len(timeline):  
            k+=1
        if k+1 == len(timeline) and timeline[k].e < feval_history[j,0]:
            # Initialize the array on the first run
            if feval_history[j,3]==0:
                feval_history[j,1]=timeline[k].f
                feval_history[j,2]=0.0
                feval_history[j,3]+=1
            else:
                old_mean=feval_history[j,1]
                feval_history[j,3]+=1
                feval_history[j,1]=feval_history[j,1]+(timeline[k].f-feval_history[j,1])/feval_history[j,3]
                feval_history[j,2]=feval_history[j,2]+(timeline[k].f-old_mean)*(timeline[k].f-feval_history[j,1])                
            break 
        else:
            # Initialize the array on the first run
            if feval_history[j,3]==0:
                feval_history[j,1]=timeline[k-1].f
                feval_history[j,2]=0.0
                feval_history[j,3]+=1
            else:    
                old_mean=feval_history[j,1]
                feval_history[j,3]+=1
                feval_history[j,1]=feval_history[j,1]+(timeline[k-1].f-feval_history[j,1])/feval_history[j,3]
                feval_history[j,2]=feval_history[j,2]+(timeline[k-1].f-old_mean)*(timeline[k-1].f-feval_history[j,1])
            
#    op.Plot_Feval_Hist(data=feval_history)
    
# Calculate averages and standard deviations
tmp=[]
for i in range(len(history[-1].d)):
    if DCS_on==False and ZDCS_on==False and BDCS_on==False and DG_on==False:
        tmp.append(sum(c.d[i] for c in history)/float(len(history)))
averages=csl.Event(sum(c.g for c in history)/float(len(history)),sum(c.e for c in history)/float(len(history)), \
            sum(c.f for c in history)/float(len(history)),tmp)

tmp=[]
if max_iter >1:
    for i in range(len(history[-1].d)):
        if  DCS_on==False and ZDCS_on==False and BDCS_on==False and DG_on==False:
            tmp.append(m.sqrt(sum([(c.d[i] - averages.d[i])**2 for c in history])/(len(history) - 1)))
    std_dev=csl.Event(m.sqrt(sum([(c.g - averages.g)**2 for c in history])/(len(history) - 1)),\
                  m.sqrt(sum([(c.e - averages.e)**2 for c in history])/(len(history) - 1)), \
                  m.sqrt(sum([(c.f - averages.f)**2 for c in history])/(len(history) - 1)),tmp)
else:
    tmp=np.zeros(len(history[-1].d))
    std_dev=csl.Event(0,0,0.0,tmp)
    
# Trim empty feval histories 
if feval_history[-1,3]==0:
    feval_history=np.array([feval_history[tmp,:] for tmp in \
                       range(len(feval_history[0:(np.argmin(feval_history[:,3]))]))])
# Compute relative error (%Diff) for feval histories
if S.of==0.0:
    feval_history[:,1]=feval_history[:,1]*100
else:
    feval_history[:,1]=abs((feval_history[:,1]-S.of)/S.of*100)
# Compute standard deviation for feval histories
for i in range(0,len(feval_history)):
    if feval_history[i,2]!=0.0:
        feval_history[i,2]=np.sqrt(feval_history[i,2]/(feval_history[i,3]-1))*100

# Output average results to the user
debug=True
if debug==True:
    print "\nThe Average Optimized Solution:"
    print "================================"
    print "Design:"
    for i in range(len(averages.d)):
        print "   var %d: %4.6f $\mypm$ %4.5f " %(i+1,averages.d[i],std_dev.d[i])
    print "Fitness: %4.6f $\mypm$ %4.5f " %(averages.f,std_dev.f)
    print "Funct Evals: %d $\mypm$ %d " %(averages.e,std_dev.e)
    print "Generations: %3.1f $\mypm$ %3.1f "  %(averages.g,std_dev.g)
    if S.of==0.0:
        print "The performance metric is %4.1f" %(averages.f*(averages.e+3*std_dev.e))
    else:
        print "The performance metric is %4.1f" %(abs((averages.f-S.of)/S.of)*(averages.e+3*std_dev.e))

#Determine the best values obtained
best=min(history,key=attrgetter('f'))

# Output best result to the user
debug=True
if debug==True:
    print "\nThe Best Optimized Solution:"
    print "================================"
    print "Design:"
    for i in range(len(averages.d)):
        print "   var %d: %4.6f " %(i+1,best.d[i])
    print "Fitness: %4.6f " %best.f
    print "Funct Evals: %d " %best.e
    print "Generations: %d "  %best.g
    if S.of==0.0:
        print "The performance metric is %4.1f" %(best.f*best.e)
    else:
        print "The performance metric is %4.1f" %(abs((best.f-S.of)/S.of)*best.e)
        
#Plot the optimization process
#if  DCS_on==False and ZDCS_on==False and BDCS_on==False and DG_on==False:
#    op.Plot_Vars(timeline,low_bounds=opt_params.lb,up_bounds=opt_params.ub,title=opt_params.pt,label=opt_params.l)
fevals=[tmp.e for tmp in history]
if max_iter >1:
    op.Plot_Hist(fevals,title=opt_params.ht)
#op.Plot_Feval_Hist(data=feval_history)

Run #0
Fitness Convergence
Program execution time was 1.634504.
Run #1
Fitness Convergence
Program execution time was 0.577079.
Run #2
Fitness Convergence
Program execution time was 0.466214.
Run #3
Fitness Convergence
Program execution time was 0.430167.
Run #4
Fitness Convergence
Program execution time was 0.492438.
Run #5
Fitness Convergence
Program execution time was 0.866904.
Run #6
Fitness Convergence
Program execution time was 0.479481.
Run #7
Fitness Convergence
Program execution time was 0.396207.
Run #8
Fitness Convergence
Program execution time was 1.012969.
Run #9
Fitness Convergence
Program execution time was 0.422767.
Run #10
Fitness Convergence
Program execution time was 0.436025.
Run #11
Fitness Convergence
Program execution time was 0.588632.
Run #12
Fitness Convergence
Program execution time was 0.570557.
Run #13
Fitness Convergence
Program execution time was 1.276016.
Run #14
Fitness Convergence
Program execution time was 0.976965.
Run #15
Fitness Convergence
Program

In [ ]:
print 'np.array([',
for i in range(0,len(feval_history)-1,1):
    print '[',feval_history[i,0],',',feval_history[i,1],',',feval_history[i,2],',',feval_history[i,3],'],',
print '[',feval_history[-1,0],',',feval_history[-1,1],',',feval_history[-1,2],',',feval_history[-1,3],']])'

In [ ]:
print np.asarray(fevals)
fevals_sa=np.array([5238,1559,2817,3526,3416,4152,2206,4245,3156,10541,4180,8432,8730,788,21581,33875,181,7497,10765,10356,13892,1464,11292,852,3102,14327,11751,10614,6644,5555,10032,11013,497,2561,12933,11375,518,16274,5350,548,11229,7942,2734,14784,2931,900,2321,5614,3524,4352,519,440,4724,2815,185,12976,1729,3880,3837,5816,8181,235,8843,464,516,5871,776,3091,1487,8701,2428,13321,1416,450,10528,10877,3704,15275,13844,1482,2233,12924,22809,1886,17178,3604,5780,7805,16553,16035,8262,2168,11495,4708,29899,270,3082,13516,6173,2082,3228,26509,10969,7620,7338,19709,3391,20966,3466,1664,5640,146,18900,1051,4302,12719,5008,1230,20339,4284,1077,10217,3022,2079,13399,8788,15191,8916,18181,2206,8424,13546,1853,6575,2497,10146,3416,3752,1215,617,4827,7514,1542,15413,623,13647,4351,8758,4714,565,8811,4506,1104,3203,910,2589,7751,853,13087,2690,11162,4630,11260,4641,2950,1806,1245,9654,218,5882,9724,4935,11808,3156,1541,240,6712,1495,8407,16436,11959,4825,1488,3310,5670,13924,14795,5077,7557,278,8377,1151,3725,23145,1540,19840,3423,7192,182,15090,1752,9963,27604,2140,2528,5210,11251,10318,14631,727,849,5221,1186,1777,5951,3280,9678,14095,16917,5966,7607,508,9909,1830,11126,22264,11875,12423,10171,467,1443,7163,1418,7074,2047,12614,13642,1480,467,13035,1808,4272,2849,1486,7382,2904,187,3764,8881,12350,8842,11332,13676,7266,7049,1100,4946,1259,2081,177,266,12580,7719,10423,1743,21977,11220,3780,3282,6753,7432,3801,845,3454,9555,4033,3042,2571,860,189,822,4179,12620,2740,8534,1933,6011,7410,4081,2548,12448,3761,1207,2242,16559,180,123,6490,21519,187,5040,1075,14680,2408,14243,11285,2096,801,7385,20283,1804,5565,168,177,1169,3047,10511,6920,11486,9818,1341,1224,6253,2155,8620,1482,16551,13096,1487,2894,10638,2610,1108,11471,6242,8306,18901,2745,235,11186,10598,6738,2463,1171,1158,3531,2557,20031,6640,1509,928,1072,1504,1416,8688,13121,11779,2819,4109,4981,12213,4446,9152,2518,3381,801,3973,9006,881,1490,4515,2453,2849,2799,6137,5850,387,8509,7797,9408,3129,1961,14542,16877,4387,1507,14145,14791,13986,1176,1523,10289,10404,10555,5804,14445,1843,611,2486,12849,1406,1424,1296,1354,11575,15491,6910,644,11406,201,15140,5246,12448,2117,4998,14934,8828,7058,3394,3100,10526,2932,2987,771,4281,4529,548,159,1763,1829,570,9929,6003,2602,1743,11276,1495,8218,2241,575,5392,3120,630,16880,13480,4797,3471,6937,4705,4550,12185,5786,3093,5426,5602,2783,2244,9728,17860,6228,4651,1162,478,6152,446,5676,2812,1727,2526,680,1175,2621,15396,7618,258,26964,205,6425,9067,3044,7175,567,8013,15140,10895,2389,5945,8057,1505,609,10301,183,9313,3649,19532,1580,12922,17984,8191,5101,10933,17647,2377,4775,15259,5170,9459,7996,4065,6290,3250,18144,7907,2439,1143,1194,5679,11949,1435,4723,24608,632,2588,1117,4014,6137,2011,2923,10183,1403,3766,4423,11525,8303,10472,11815,3106,17964,1247,2733,514,16436,8255,9979,495,10189,5637,8987,204,2510,9937,580,12268,14791,5330,228,3419,8556,11264,10912,16607,10947,10058,1165,13114,4435,2197,5680,4489,2068,504,2101,1484,3925,2251,1466,2114,6038,3517,1769,9388,22773,2037,4617,1689,1716,2406,181,3754,215,1125,15972,478,3321,13025,6010,2695,13233,4255,10434,3526,17869,748,4194,7953,4882,2401,1783,2121,1214,17426,2741,10939,10623,158,646,597,5114,13098,1463,3486,1241,4020,3263,19729,10982,834,1984,7154,2897,7695,8149,480,4840,500,13207,2809,4397,1158,12157,806,610,15135,397,2840,8636,1832,11215,1750,2916,8365,1524,2037,1543,1368,4420,7806,1467,3056,5773,4190,11963,848,1732,13152,15071,10848,12540,489,3179,11950,9816,1442,876,16878,3232,18647,3024,11423,1170,4044,1281,1717,3704,8525,7689,1632,10906,10947,1553,842,3410,19792,10223,10428,484,588,10499,4122,2483,2685,201,3430,8908,11617,782,5436,13070,7250,4390,15560,793,6455,13012,2167,3276,5096,5469,153,425,7912,1058,12555,191,211,4592,4207,268,10123,5342,4067,18088,1416,4310,16645,578,1604,9217,3514,13589,422,23209,6281,14466,2467,1101,3692,6371,18763,11239,3663,5406,491,3102,9377,11083,1215,223,213,203,472,6608,12531,14478,1625,1817,5587,480,6751,8796,837,1897,3833,5682,862,14714,3517,12947,469,15108,1057,859,5091,1212,1292,4126,11877,2040,20828,3469,2996,3679,6173,854,13293,2204,1882,396,1804,15969,2817,3770,603,5480,1105,2734,1978,2116,3566,834,3028,1612,9141,3403,1148,1494,2455,8944,2903,798,8604,1566,931,1209,4592,997,4891,1521,1929,853,7472,917,1405,812,10748,12374,9579,16117,150,1810,1845,515,208,5777,489,1530,4198,14846,2384,9130,2261,18337,1242,6522,3135,4007,811,829,5234,10452,3166,9221,3815,14020,9217,12777,267,3346,492,10482,3022,3599,7846,1114,1489,3825,1175,1378,5883,11869,12792,3466,2922,6499,8404,7497,17463,2560,10923,977,11696,4245,466,5411,12930,10598,5539,865,3881,855,3841,16362,3456,1497,5797,6821,802,894,2175,5318,2877,3614,14600,2038,3792,8653,189,18744,12735,8257,1597,18335,2827,3571,10583,137,2310,9460,3555,510,11058,2565,13065,6327,11542,2504,2734,3725,6767,22992,11720,4595,10179,3682,6393,4064,1485,11309,4344,769,19731,6463,5793,10974,1804,6870,479,7142,1760,10361,3408,3680,12037,8408,10443,1562,21544,528,13234,2703,10900,4909,3720,7411,3525,6944,8929,6152,8100,10009,15069,9047,1504,20073,2920,815,17845,2083,499,2512,9701,13120,10500,6136,13150,7606,4330,3294,12156,12576])
print fevals_sa

In [ ]:
from ObjectiveFunctions import Chemical_Process_Obj
Chemical_Process_Obj()